In [8]:
import networkx as nx
import pandas as pd
import community
import numpy as np
import utils

In [9]:
def load_graph(filename):
    df = pd.read_csv(filename, header=None, names=['source', 'target', 'weight'])
    G = nx.from_pandas_edgelist(df, edge_attr='weight', create_using=nx.Graph())
    return G

In [10]:
def add_inference_edges(G, edge_list):
    edges = pd.read_csv(edge_list, header = None)
    edges.columns = ['src', 'dst', 'weight']
    edge_tuples = []
    for i in range(edges.shape[0]):
        src = edges.iloc[i, 0]
        dst = edges.iloc[i, 1]
        weight = edges.iloc[i, 2]
        edge_tuples.append((src, dst, weight))
    G.add_weighted_edges_from(edge_tuples)
    return G

In [11]:
def write_out_results(reverse_comms, filename):
    # Write results to file
    f = open('results/' + filename, 'a')
    for cmty, nodes in reverse_comms.items():
        f.write("Community %d:\n" % cmty)
        for node in nodes:
            f.write(nodes_mapping[node] + "\n")
        f.write("\n")

In [12]:
G = load_graph('data/reddit_nodes_weighted_full.csv')
print('OG edges:', len(G.edges()))
G = add_inference_edges(G, 'new_edges/linreg_rolx_edges_to_add.csv')
print('New edges: ', len(G.edges()))

OG edges: 309667
New edges:  372285


In [ ]:
R_t, G_1, layers_1 = utils.hicode(G,1)
print("hicode 1",R_t)

In [ ]:
R_t, G_2, layers_2 = utils.hicode(G,2)
print("hicode 2",R_t)

In [ ]:
R_t, G_3, layers_3 = utils.hicode(G,3)
print("hicode 3",R_t)

In [ ]:
nodes_mapping = utils.load_nodes_mapping()
partitions = []
for num_layer, layer in enumerate(layers_2):
    print("community_count",len(layer))
    revised_community_count = 0
    for i,subgraph in enumerate(layer):
        if len(subgraph.nodes) > 100:
            revised_community_count += 1
    print("revised_community_count",revised_community_count)
    
    partition = utils.layer_to_partition(layer,G_weighted)
    print("layer number ", num_layer + 1)
    print(utils.modularity(partition,G_weighted))
    print(utils.modularity(partition,G_2))
    partitions.append(partition)
    reverse_comms = utils.reverse_dict(partition)
    utils.write_results_to_file(reverse_comms,nodes_mapping,"layer_"+str(num_layer + 1))